In [13]:
import numpy as np
import os
import time

import cv2
import torch

class SuperPointNet(torch.nn.Module):
  """ Pytorch definition of SuperPoint Network. """
  def __init__(self):
    super(SuperPointNet, self).__init__()
    self.relu = torch.nn.ReLU(inplace=True)
    self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2)
    c1, c2, c3, c4, c5, d1 = 64, 64, 128, 128, 256, 256
    # Shared Encoder.
    self.conv1a = torch.nn.Conv2d(1, c1, kernel_size=3, stride=1, padding=1)
    self.conv1b = torch.nn.Conv2d(c1, c1, kernel_size=3, stride=1, padding=1)
    self.conv2a = torch.nn.Conv2d(c1, c2, kernel_size=3, stride=1, padding=1)
    self.conv2b = torch.nn.Conv2d(c2, c2, kernel_size=3, stride=1, padding=1)
    self.conv3a = torch.nn.Conv2d(c2, c3, kernel_size=3, stride=1, padding=1)
    self.conv3b = torch.nn.Conv2d(c3, c3, kernel_size=3, stride=1, padding=1)
    self.conv4a = torch.nn.Conv2d(c3, c4, kernel_size=3, stride=1, padding=1)
    self.conv4b = torch.nn.Conv2d(c4, c4, kernel_size=3, stride=1, padding=1)
    # Detector Head.
    self.convPa = torch.nn.Conv2d(c4, c5, kernel_size=3, stride=1, padding=1)
    self.convPb = torch.nn.Conv2d(c5, 65, kernel_size=1, stride=1, padding=0)
    # Descriptor Head.
    self.convDa = torch.nn.Conv2d(c4, c5, kernel_size=3, stride=1, padding=1)
    self.convDb = torch.nn.Conv2d(c5, d1, kernel_size=1, stride=1, padding=0)

  def forward(self, x):
    """ Forward pass that jointly computes unprocessed point and descriptor
    tensors.
    Input
      x: Image pytorch tensor shaped N x 1 x H x W.
    Output
      semi: Output point pytorch tensor shaped N x 65 x H/8 x W/8.
      desc: Output descriptor pytorch tensor shaped N x 256 x H/8 x W/8.
    """
    # Shared Encoder.
    x = self.relu(self.conv1a(x))
    x = self.relu(self.conv1b(x))
    x = self.pool(x)
    x = self.relu(self.conv2a(x))
    x = self.relu(self.conv2b(x))
    x = self.pool(x)
    x = self.relu(self.conv3a(x))
    x = self.relu(self.conv3b(x))
    x = self.pool(x)
    x = self.relu(self.conv4a(x))
    x = self.relu(self.conv4b(x))
    # Detector Head.
    cPa = self.relu(self.convPa(x))
    semi = self.convPb(cPa)
    # Descriptor Head.
    cDa = self.relu(self.convDa(x))
    desc = self.convDb(cDa)
    dn = torch.norm(desc, p=2, dim=1) # Compute the norm.
    desc = desc.div(torch.unsqueeze(dn, 1)) # Divide by norm to normalize.
    return semi, desc



In [14]:
class Config:
    def __init__(self):
        self.weights_path = './weights/superpoint_v1.pth'
        self.cuda = False
        self.H = 480
        self.W = 640
        self.conf_thresh = 0.015 #Detector confidence threshold (default: 0.015).
        self.nms_dist = 10 # #Non Maximum Suppression (NMS) distance (default: 4). 非最大值抑制半径

cfg = Config()


In [15]:
net = SuperPointNet()
net.load_state_dict(torch.load(cfg.weights_path,map_location=lambda storage, loc: storage))
net.eval()

def export_pytorch_jit_model(net,cfg,name):
    trace_model = torch.jit.trace(net, torch.Tensor(1, 1, cfg.H, cfg.W))
    trace_model.save(f'./weights/{name}_{cfg.W}x{cfg.H}.pt')

# export_pytorch_jit_model(net,cfg,'sp')

In [16]:
from rknn.api import RKNN
# Create RKNN object
rknn = RKNN(verbose=True)

input_size_list = [[1, 1, cfg.H, cfg.W]]

# Pre-process config
print('--> Config model')
rknn.config( target_platform='rk3588')
print('done')

# Load model
print('--> Loading model')
ret = rknn.load_pytorch(model=f'./weights/sp_{cfg.W}x{cfg.H}.pt', input_size_list=input_size_list)
if ret != 0:
    print('Load model failed!')
    exit(ret)
print('done')

# Build model
print('--> Building model')
ret = rknn.build(do_quantization=False, dataset='./dataset.txt')
if ret != 0:
    print('Build model failed!')
    exit(ret)
print('done')

# Export rknn model
print('--> Export rknn model')
ret = rknn.export_rknn(f'./sp_{cfg.W}x{cfg.H}.rknn')
if ret != 0:
    print('Export rknn model failed!')
    exit(ret)
print('done')

W __init__: rknn-toolkit2 version: 1.6.0+81f21f4d


--> Config model
done
--> Loading model


Loading : 100%|████████████████████████████████████████████████████| 26/26 [00:00<00:00, 246.17it/s]
W load_pytorch: The config.mean_values is None, zeros will be set for input 0!
W load_pytorch: The config.std_values is None, ones will be set for input 0!
W build: The dataset='./dataset.txt' is ignored because do_quantization = False!
I base_optimize ...
I base_optimize done.
I 
I fold_constant ...


done
--> Building model


I fold_constant done.
I 
I correct_ops ...
I correct_ops done.
I 
I fuse_ops ...
I fuse_ops results:
I     convert_unsqueeze_to_reshape: remove node = ['102_Unsqueeze'], add node = ['102_Unsqueeze_2reshape']
I     convert_reduce_L2_to_pow_sum_pow: remove node = ['dn.1_ReduceL2'], add node = ['input.1_pow_2', 'input.1_sum', 'input.1_pow_0.5']
I     convert_reduce_sum_to_conv: remove node = ['input.1_sum'], add node = ['input.1_sum_2conv', 'input.1_sum_2conv_after']
I     unsqueeze_to_4d_pow: remove node = [], add node = ['input.1_pow_0.5_0_unsqueeze0', 'input.1_pow_0.5_0_unsqueeze1']
I     fuse_two_reshape: remove node = ['input.1_sum_2conv_after', 'input.1_pow_0.5_0_unsqueeze1']
I     fold_constant ...
I     fold_constant done.
I fuse_ops done.
I 
I sparse_weight ...
I sparse_weight done.
I 
E build: Catch exception when building RKNN model!
E build: Traceback (most recent call last):
E build:   File "rknn/api/rknn_base.py", line 2034, in rknn.api.rknn_base.RKNNBase.build
E build:   Fi

Build model failed!
done
--> Export rknn model
Export rknn model failed!
done
